In [ ]:
#!pip install yahoofinancials
#pip install yfinance

In [1]:
from tqdm import tqdm
from datetime import datetime, timedelta
import re
import os
import pandas as pd

In [2]:
tickers_SP500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_list()
tickers_Nasdaq100 = pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100#External_links')[4]["Ticker"].to_list()

In [3]:
import yfinance as yf
tickers = ['^IXIC','^GSPC','^DJI','AAPL','TQQQ']

for ticker in tqdm(tickers, desc="Processing", unit="tickers"):
    stock = yf.Ticker(ticker)
    df = stock.history(start="2004-01-01", end="2024-01-01", interval="1d")
    df['ticker'] = ticker
    
    df.to_csv(f'yf/{ticker}.csv')

Processing: 100%|██████████| 5/5 [00:07<00:00,  1.60s/tickers]


In [5]:
from yahoofinancials import YahooFinancials
tickers = ['^IXIC','^GSPC','^DJI']

for ticker in tqdm(tickers, desc="Processing", unit="tickers"):
    yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_historical_price_data(start_date='2004-01-01', 
                                                      end_date='2024-01-01', 
                                                      time_interval='daily')
    df = pd.DataFrame(data[ticker]['prices'])
    df['ticker'] = ticker
    df.to_csv(f'yf/{ticker}.csv', index = False)

Processing: 100%|██████████| 3/3 [00:03<00:00,  1.22s/tickers]


In [6]:
df

,date,high,low,open,close,volume,adjclose,formatted_date,ticker
0,1073053800,10527.030273,10384.299805,10452.740234,10409.849609,168890000,10409.849609,2004-01-02,^DJI
1,1073313000,10544.070312,10411.849609,10411.849609,10544.070312,221290000,10544.070312,2004-01-05,^DJI
2,1073399400,10549.179688,10499.849609,10543.849609,10538.660156,191460000,10538.660156,2004-01-06,^DJI
3,1073485800,10539.459961,10466.290039,10535.459961,10529.030273,225490000,10529.030273,2004-01-07,^DJI
4,1073572200,10592.589844,10530.070312,10530.070312,10592.440430,237770000,10592.440430,2004-01-08,^DJI
...,...,...,...,...,...,...,...,...,...
5028,1703255400,37534.519531,37268.878906,37349.269531,37385.968750,252970000,37385.968750,2023-12-22,^DJI
5029,1703601000,37617.988281,37371.828125,37405.898438,37545.328125,212420000,37545.328125,2023-12-26,^DJI
5030,1703687400,37683.699219,37488.601562,37518.621094,37656.519531,245530000,37656.519531,2023-12-27,^DJI
5031,1703773800,37778.851562,37650.980469,37661.519531,37710.101562,199550000,37710.101562,2023-12-28,^DJI


In [9]:
tickers = tickers_Nasdaq100

for ticker in tqdm(tickers, desc="Processing", unit="tickers"):
    
    ticker = re.sub(r'\.', '-', ticker)
   
    yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_historical_price_data(start_date='2004-01-01', 
                                                      end_date='2024-01-01', 
                                                      time_interval='daily')
    df = pd.DataFrame(data[ticker]['prices'])
    df['ticker'] = ticker
    df.to_csv(f'yf/{ticker}.csv', index = False)


print("Download completed!")

Processing: 100%|██████████| 101/101 [16:44<00:00,  9.95s/tickers]

Download completed!


In [ ]:
tickers = tickers_SP500

current_date = datetime.now()
current_date_str = current_date.strftime("%Y-%m-%d")

for ticker in tqdm(tickers, desc="Processing", unit="tickers"):
    
    ticker = re.sub(r'\.', '-', ticker)
    
    folder_path = 'yf'
    file_name = f'{ticker}.csv'
    file_path = os.path.join(folder_path, file_name)
    
    if os.path.exists(file_path):
        #update csv
        df = pd.read_csv(file_path)
        last_date_str = df.loc[df.index[-1],'formatted_date']

        start_date = datetime.strptime(last_date_str, "%Y-%m-%d") + timedelta(1)
        start_date_str = start_date.strftime("%Y-%m-%d")

        if start_date < (current_date+timedelta(-1)):                
            yahoo_financials = YahooFinancials(ticker)
            data = yahoo_financials.get_historical_price_data(start_date = start_date_str, 
                                                              end_date = current_date_str, 
                                                              time_interval = 'daily')
            if data[ticker] != None:
                df_update = pd.DataFrame(data[ticker]['prices'])
                df_update['ticker'] = ticker
                #df=pd.concat([df, df_update], ignore_index=True,axis=0)
                #df.to_csv(file_path, index = False)
                df_update.to_csv(file_path, header=None, index=False, mode='a')
        
    else:
        #create csv
        start_date_str = '2004-01-01'
        current_date_str = '2024-01-01'
          
        yahoo_financials = YahooFinancials(ticker)
        data = yahoo_financials.get_historical_price_data(start_date=start_date_str, 
                                                          end_date=current_date_str, 
                                                          time_interval='daily')
        df = pd.DataFrame(data[ticker]['prices'])
        df['ticker'] = ticker
        df.to_csv(file_path, index = False)
            

print("Download completed!")

In [90]:
ticker = "aapl"
stock = yf.Ticker(ticker)
print(dir(stock))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_analysis', '_base_url', '_data', '_download_options', '_earnings', '_earnings_dates', '_expirations', '_fast_info', '_fetch_ticker_tz', '_financials', '_fix_bad_stock_split', '_fix_missing_div_adjust', '_fix_prices_sudden_change', '_fix_unit_mixups', '_fix_unit_random_mixups', '_fix_unit_switch', '_fix_zeroes', '_fundamentals', '_get_ticker_tz', '_history', '_history_metadata', '_history_metadata_formatted', '_holders', '_isin', '_news', '_options2df', '_quote', '_reconstruct_intervals_batch', '_reconstruct_start_interval', '_shares', '_tz', 'actions', 'analyst_price_target', 'balance_sheet', 'balancesheet', 'basic_info', 'calendar', 'capital_ga